
## NBA project

Here you will find a little project about data analysis. Particularly, this project contains data manipulation, data visualization, OLS, Logit and MEM (Mixed Effects Models). I´m using an `nba` database which contains 5313 players from 1947-2025. You can find more details about this dataset [here](https://www.kaggle.com/datasets/flynn28/v2-nba-player-database/data).

I will be working with this variables:

`Name`: Players name

`Position`: Players position(s)

`Height`: Height of player (inches)

`Weight`: Weight of player (lbs)

`School`: School(s) player attended

`Active`: If player is currently playing (True or False)

`G`: amount of games played by player

`PTS`: average points scored by player per game

`TRB`: average rebounds by player per game

`AST`: average assists per game

## Research exercise

For this project, I will be analyzing the relationship between being a Center (here I will include those who can play two positions, for example Forward-Center or Center-Forward ) and scoring performance. Also, some predictors will be included, such as Games played, Height, Weight, average total asists per game (AST), average total rebounds per game (TRB).

## Data and libraries

So, let's get into the data! We will be working with the following libraries. Then we load our NBA data.

In [1]:

library(dplyr)
library(ggplot2)
library(scales)
library(lme4)
library(misty)
library(texreg)
library(ggrepel)
library(tidyr)
library(lattice)
library(gridExtra)
library(ggthemes)

nba<- read.csv("D:/RSTUDIOWD/papers/nba-proyect/NBA_PLAYERS.csv")

head(nba,10)


Adjuntando el paquete: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Cargando paquete requerido: Matrix

|-------------------------------------|
| misty 0.7.2 (2025-05-20)            |
| Miscellaneous Functions T. Yanagida |
|-------------------------------------|

Version:  1.39.4
Date:     2024-07-23
Author:   Philip Leifeld (University of Manchester)

Consider submitting praise using the praise or praise_interactive functions.
Please cite the JSS article in your publications -- see citation("texreg").


Adjuntando el paquete: 'tidyr'


The following object is masked from 'package:texreg':

    extract


The following objects are masked from 'package:Matrix':

    expand, pack, unpack



Adjuntando el paquete: 'gridExtra'


The following object is masked from 'package:dplyr':

    combine




,Name,Debut,Final,Position,Height,Weight,Birthday,School,HOF,Active,G,PTS,TRB,AST,FG.,FG3.,FT.,eFG.,PER,WS
,<chr>,<int>,<int>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Alaa Abdelnaby,1991,1995,"['Forward', 'Center']",82,240,"June 24, 1968",['Duke'],False,False,256,5.7,3.3,0.3,50.2,0.0,70.1,50.2,13.0,4.8
2,Zaid Abdul-Aziz,1969,1978,"['Center', 'Forward']",81,235,"April 7, 1946",['Iowa State'],False,False,505,9.0,8.0,1.2,42.8,NA,72.8,NA,15.1,17.5
3,Kareem Abdul-Jabbar,1970,1989,['Center'],86,225,"April 16, 1947",['UCLA'],True,False,1560,24.6,11.2,3.6,55.9,5.6,72.1,55.9,24.6,273.4
4,Mahmoud Abdul-Rauf,1991,2001,['Guard'],73,162,"March 9, 1969",['LSU'],False,False,586,14.6,1.9,3.5,44.2,35.4,90.5,47.2,15.4,25.2
5,Tariq Abdul-Wahad,1998,2003,['Forward'],78,223,"November 3, 1974","['Michigan', ' San Jose State']",False,False,236,7.8,3.3,1.1,41.7,23.7,70.3,42.2,11.4,3.5
6,Shareef Abdur-Rahim,1997,2008,['Forward'],81,225,"December 11, 1976",['California'],False,False,830,18.1,7.5,2.5,47.2,29.7,81.0,47.9,19.0,71.2
7,Tom Abernethy,1977,1981,['Forward'],79,220,"May 6, 1954",['Indiana'],False,False,319,5.6,3.2,1.2,49.2,0.0,74.7,49.2,12.9,13.4
8,Forest Able,1957,1957,['Guard'],75,180,"July 27, 1932",['Western Kentucky'],False,False,1,0.0,1.0,1.0,0.0,NA,NA,NA,-41.5,0.0
9,John Abramovic,1947,1948,['Forward'],75,195,"February 9, 1919",['Salem University'],False,False,56,9.5,NA,0.7,23.7,NA,68.6,NA,NA,-1.9


## Variable processing in R

First, we clear our NA cases


In [ ]:
nba<- nba %>% 
  na.omit()

Now, let's take a look at the variables in the dataset

In [ ]:
colnames(nba)

For the Schools, we display the 10 schools with most cases

In [ ]:
colnames(nba)
nba %>% 
count(School, sort = T) %>% 
  head(10)

Here we observe that the variable with most cases is not labeled, but is not a missed value either. This means that these players did not come from any School. We label this situation as \`street\`.

In [ ]:
nba <- nba %>%
  mutate(School = ifelse(School == "" | is.na(School), "street", School))

For \`Height\` and \`Weight\` variables, we transform them to a real metric system :D

In [ ]:
nba<- nba %>% 
  mutate(
    Height_cm = Height * 2.54,
    Weight_kg = Weight * 0.453592,
    total_pts_carrer = PTS * G
    )

nba %>% 
  summarise(
    mean_pts = mean(PTS),
    max_pts = max(PTS),
    sd_pts = sd(PTS),
    avg_height = mean(Height_cm),
    avg_weight = mean(Weight_kg)
  )

For \`Position\` variable, we display its values and then filter the variable to include only those cases in which players are listed as Center, Forward-Center or Center-Forward.

In [ ]:
nba %>% 
  count(Position)

nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']")) %>% 
  group_by(Position) %>% 
  summarise(AVG_PTS = mean(PTS))

Now we can explore our Centers

In [ ]:
nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']"),
         Active == "True") %>%  
  group_by(Position) %>% 
  slice_max(order_by = PTS, n = 10) %>% 
  select(Position, Name, PTS, Active, Height_cm, Weight_kg) %>% 
  arrange(Position, desc(PTS))


Top 3 Centers

In [ ]:
nba %>% 
  filter(Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']"),
         Active == "True") %>%  
  group_by(Position) %>% 
  slice_max(order_by = PTS, n = 3) %>% 
  select(Position, Name, PTS, Active, Height_cm, Weight_kg, G,total_pts_carrer) %>% 
  arrange(Position, desc(PTS))

Before we jump into the inference analysis, we need to do some changes first:

-   Rename the variable \`G\` to \`games\`

-   Create a new variable called \`PTSCENTER\`, which contains the average points scored by active Centers in 2025

-   Change the reference category for Position variable to "\['Center'\]"

In [ ]:
nba<- nba %>% 
  mutate(
    games = G,
    PTSCENTER = case_when(
      Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']") & Active == "True" ~ PTS
    )
  )

We create a new dataset for active Centers

In [ ]:
nba_centers <- nba %>%
  filter(!is.na(PTSCENTER))

## Inference Analysis

For this part of the exercise, we estimate a linear regression model (OLS), which says as follows:

$$
AvgPointsScored_i = \beta_0 + \beta_1  Position_i + \beta_2Games_i + \beta_3Height_{cm_i} + \beta_4 Weight_{cm_i} + \\ \beta_5AvgAsist_i +  \beta_6AvgRebounds_i + \varepsilon_i
$$

We estimate model 1 and its fitted values

In [ ]:
m1<- lm(PTSCENTER ~ Position + games +Height_cm + Weight_kg + AST + TRB, data = nba_centers )
screenreg(m1)

nba_centers$fittedvalues <- predict(m1)

After that, we obtain the 10 highest fitted values and then we generate the plot

In [ ]:
nba_centers %>% 
  select(Name, PTS, fittedvalues, Position) %>% 
  slice_max(order_by = fittedvalues, n = 10) 


In [ ]:
nba_centers %>%
  ggplot(aes(x = fittedvalues, y = PTS)) +
  geom_point(alpha = 0.6, color = "steelblue") +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "red") +
  labs(
    title = "Comparación entre puntos reales y predichos",
    x = "Puntos predichos (fittedvalues)",
    y = "Puntos reales (PTS)"
  ) +
  theme_bw()

## Logit Model

For this part, we estimate a logit model. First things first, we obtain the average rebounds from the entire sample. Then we get the max and min rebounds values.

In [ ]:
nba_centers %>% 
  summarise(
    avg_rb = mean(TRB),
    max = max(TRB),
    min = min(TRB)
  )

In the following step, we create a new variable called \`rb_dummy\` in which returns 1 if the player have an average of 6 or greater rebounds per game, and 0 if the total rebounds are less than 6.

In [ ]:
nba_centers <- nba_centers %>% 
  mutate(
    rb_dummy = case_when(
      TRB >= 6 ~ 1,
      TRB < 6 ~ 0
    )
  )

We estimate the logit model

In [ ]:
m2<- glm(rb_dummy ~  PTSCENTER + Position + games +Height_cm + Weight_kg + AST, data = nba_centers,
         family = binomial (link = "logit") )
screenreg(m2)

INTERPRETAR

## Multilevel model (MEM)

Finally, we estimate a multilevel model. In this part, we expand the sample to include all positions and players in the data base whether actives or not. Lastly, for the level-2 variables, we select the top 5 schools with the highest players in order to simplify the exercise.

In [ ]:
nba<- nba %>% 
  mutate(
    CENTERS = Position %in% c("['Center', 'Forward']", "['Center']", "['Forward', 'Center']")
    )
  
top5_schools <- nba %>%
  count(School, sort = TRUE) %>%
  slice_max(n, n = 5) %>%
  pull(School)
  
 

nba_top5_schools <- nba %>% 
  filter(School %in% top5_schools) 

nba_top5_schools %>% 
  count(School, sort = T)

## Centering

Our interest is to


In [ ]:
nba_top5_schools$games_cgm <- center(nba_top5_schools$games, type = "CGM")
nba_top5_schools$Height_cm_cgm <- center(nba_top5_schools$Height_cm, type = "CGM")
nba_top5_schools$Weight_kg_cgm <- center(nba_top5_schools$Weight_kg, type = "CGM")
nba_top5_schools$AST_cgm <- center(nba_top5_schools$AST, type = "CGM")
nba_top5_schools$TRB_cgm <- center(nba_top5_schools$TRB, type = "CGM")

We estimate three models. First, the

In [ ]:
mnulo<- lmer(PTS ~ 1 + (1|School), data = nba_top5_schools)

m3<- lmer(PTS ~ Position + games_cgm + Height_cm_cgm + Weight_kg_cgm + AST_cgm + TRB_cgm + (1|School)  , data = nba_top5_schools)


m4<- lmer(PTS ~ Position + games_cgm + Height_cm_cgm + Weight_kg_cgm + AST_cgm + TRB_cgm + (TRB_cgm|School)  , data = nba_top5_schools)
screenreg(list(mnulo,m3, m4), single.row = T)

Now we calculate the ICC


In [ ]:
ICC = 0.93/(0.93+29.32) * 100
print(ICC)


ANOVA

In [ ]:
anova(m3,m4)

qqmath graph

In [ ]:
qqmath(ranef(mnulo, condVar = TRUE))

Now, we plot model 3

In [ ]:
ranefs_m3 <- ranef(m3)$School
ranefs_m3$School <- rownames(ranefs_m3)
colnames(ranefs_m3) <- c("intercept_ranef", "School")

fixefs_m3 <- fixef(m3)
intercept_fixed <- fixefs_m3["(Intercept)"]
slope_fixed <- fixefs_m3["TRB_cgm"]


TRB_seq <- seq(from = min(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               to = max(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               length.out = 100)

line_data_m3 <- expand.grid(
  TRB_cgm = TRB_seq,
  School = unique(nba_top5_schools$School)
)

line_data_m3 <- left_join(line_data_m3, ranefs_m3, by = "School")

line_data_m3 <- line_data_m3 %>%
  mutate(
    intercept_total = intercept_fixed + intercept_ranef,
    # Pendiente fija, igual para todas las escuelas
    slope_total = slope_fixed,
    PTS_pred = intercept_total + slope_total * TRB_cgm
  )

ggplot(nba_top5_schools, aes(x = TRB_cgm, y = PTS, color = School)) +
  geom_point(alpha = 0.6) +
  geom_line(data = line_data_m3, aes(x = TRB_cgm, y = PTS_pred, color = School), size = 1) +
  facet_wrap(~ School) +
  labs(
    title = "Fitted values m3 (random intercept, fixed slope)",
    x = "Centering TRB (Grand Mean)",
    y = "PTS",
    color = "School"
  ) +
  theme_bw()

modelo 4 plot

In [ ]:
nba_top5_schools$PTS_pred <- predict(m4, re.form = NULL)
ranefs <- ranef(m4)$School
ranefs$School <- rownames(ranefs)
colnames(ranefs) <- c("intercept_ranef", "TRB_cgm_ranef", "School")


fixefs <- fixef(m4)
intercept_fixed <- fixefs["(Intercept)"]
slope_fixed <- fixefs["TRB_cgm"]


# Crear un grid de valores centrados de TRB
TRB_seq <- seq(from = min(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               to = max(nba_top5_schools$TRB_cgm, na.rm = TRUE),
               length.out = 100)

# Expandir para cada escuela
line_data <- expand.grid(
  TRB_cgm = TRB_seq,
  School = unique(nba_top5_schools$School)
)

# Juntar con efectos aleatorios
line_data <- left_join(line_data, ranefs, by = "School")


# Calcular pendiente e intercepto total por escuela
line_data <- line_data %>%
  mutate(
    intercept_total = intercept_fixed + intercept_ranef,
    slope_total = slope_fixed + TRB_cgm_ranef,
    PTS_pred = intercept_total + slope_total * TRB_cgm
  )

ggplot(nba_top5_schools, aes(x = TRB_cgm, y = PTS, color = School)) +
  geom_point(alpha = 0.6) +
  geom_line(data = line_data, aes(x = TRB_cgm, y = PTS_pred, color = School), size = 1) +
  facet_wrap(~ School) +
  labs(
    title = "Fitted values m4 (random intercept, random slope)",
    x = "Centering TRB (Grand Mean)",
    y = "PTS",
    color = "School"
  ) +
  theme_bw()

Lastly, we obtain the intercepts and slopes of each model.

In [ ]:
# Para m3 (solo intercepto aleatorio)
ranefs_m3 <- ranef(m3)$School %>% 
  as.data.frame() %>% 
  rename(intercept_ranef = `(Intercept)`) %>% 
  mutate(School = rownames(ranef(m3)$School)) %>% 
  select(School, intercept_ranef)

fixefs_m3 <- fixef(m3)
intercept_fixed_m3 <- fixefs_m3["(Intercept)"]
slope_fixed_m3 <- fixefs_m3["TRB_cgm"]

ranefs_m3 <- ranefs_m3 %>%
  mutate(
    intercept_total = intercept_fixed_m3 + intercept_ranef,
    slope_fixed = slope_fixed_m3
  )

print("Interceptos y pendiente fija por escuela modelo m3:")
print(ranefs_m3)


In [ ]:
ranefs_m4 <- ranef(m4)$School %>% 
  as.data.frame() %>% 
  rename(
    intercept_ranef = `(Intercept)`,
    slope_ranef = TRB_cgm
  ) %>% 
  mutate(School = rownames(ranef(m4)$School)) %>% 
  select(School, intercept_ranef, slope_ranef)

fixefs_m4 <- fixef(m4)
intercept_fixed_m4 <- fixefs_m4["(Intercept)"]
slope_fixed_m4 <- fixefs_m4["TRB_cgm"]

ranefs_m4 <- ranefs_m4 %>%
  mutate(
    intercept_total = intercept_fixed_m4 + intercept_ranef,
    slope_total = slope_fixed_m4 + slope_ranef
  )

print("Interceptos y pendientes (random slopes) por escuela modelo m4:")
print(ranefs_m4)